## Вбудовування

У нашому попередньому прикладі ми працювали з високорозмірними векторами "мішків слів" довжиною `vocab_size` і явно перетворювали низькорозмірні вектори позиційного представлення у розріджене однохотне представлення. Це однохотне представлення не є ефективним з точки зору пам'яті. Крім того, кожне слово розглядається незалежно одне від одного, тому однохотні вектори не виражають семантичних подібностей між словами.

У цьому розділі ми продовжимо досліджувати набір даних **News AG**. Для початку завантажимо дані та отримаємо деякі визначення з попереднього розділу.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Що таке вбудовування?

Ідея **вбудовування** полягає в тому, щоб представляти слова за допомогою низьковимірних щільних векторів, які відображають семантичне значення слова. Пізніше ми обговоримо, як створювати змістовні вбудовування слів, але наразі розглянемо вбудовування як спосіб зменшення розмірності вектора слова.

Отже, шар вбудовування приймає слово як вхід і створює вихідний вектор заданого розміру `embedding_size`. У певному сенсі, це дуже схоже на шар `Dense`, але замість того, щоб приймати вхідний вектор у вигляді one-hot кодування, він може приймати номер слова.

Використовуючи шар вбудовування як перший шар у нашій мережі, ми можемо перейти від моделі "мішка слів" до моделі **мішка вбудовувань**, де спочатку кожне слово в тексті перетворюється на відповідне вбудовування, а потім обчислюється певна агрегатна функція для всіх цих вбудовувань, наприклад, `sum`, `average` або `max`.

![Зображення, що показує класифікатор з вбудовуванням для п’яти послідовних слів.](../../../../../translated_images/uk/embedding-classifier-example.b77f021a7ee67eee.webp)

Наша нейронна мережа класифікатора складається з таких шарів:

* Шар `TextVectorization`, який приймає рядок як вхід і створює тензор номерів токенів. Ми визначимо розумний розмір словника `vocab_size` і ігноруватимемо менш уживані слова. Вхідна форма буде 1, а вихідна форма — $n$, оскільки ми отримаємо $n$ токенів у результаті, кожен із яких містить числа від 0 до `vocab_size`.
* Шар `Embedding`, який приймає $n$ чисел і зменшує кожне число до щільного вектора заданої довжини (у нашому прикладі — 100). Таким чином, вхідний тензор форми $n$ буде перетворений на тензор $n\times 100$.
* Шар агрегації, який обчислює середнє значення цього тензора вздовж першої осі, тобто обчислює середнє значення всіх $n$ вхідних тензорів, що відповідають різним словам. Для реалізації цього шару ми використаємо шар `Lambda` і передамо в нього функцію для обчислення середнього значення. Вихідна форма буде 100, і це буде числове представлення всієї вхідної послідовності.
* Завершальний лінійний класифікатор `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


У `summary`-виводі, у колонці **output shape**, перший вимір тензора `None` відповідає розміру мініпакету, а другий — довжині послідовності токенів. Усі послідовності токенів у мініпакеті мають різну довжину. Ми обговоримо, як із цим впоратися, у наступному розділі.

А тепер давайте навчимо мережу:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Примітка**: ми створюємо векторизатор на основі підмножини даних. Це робиться для прискорення процесу, і це може призвести до ситуації, коли не всі токени з нашого тексту присутні у словнику. У такому випадку ці токени будуть ігноровані, що може призвести до трохи нижчої точності. Однак у реальному житті підмножина тексту часто дає хорошу оцінку словника.


### Робота зі змінними розмірами послідовностей

Давайте розберемося, як відбувається навчання в мініпакетах. У наведеному вище прикладі вхідний тензор має розмірність 1, і ми використовуємо мініпакети довжиною 128, тому фактичний розмір тензора становить $128 \times 1$. Однак кількість токенів у кожному реченні різна. Якщо ми застосуємо шар `TextVectorization` до одного входу, кількість токенів, що повертається, буде різною, залежно від того, як текст токенізується:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Однак, коли ми застосовуємо векторизатор до кількох послідовностей, він має створити тензор прямокутної форми, тому заповнює невикористані елементи токеном PAD (який у нашому випадку дорівнює нулю):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Ось ми можемо побачити вбудовування:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Примітка**: Щоб мінімізувати кількість заповнення, в деяких випадках має сенс сортувати всі послідовності в наборі даних у порядку зростання довжини (або, точніше, кількості токенів). Це забезпечить, що кожна мініпартія містить послідовності схожої довжини.


## Семантичні вбудовування: Word2Vec

У нашому попередньому прикладі шар вбудовування навчався відображати слова у векторні представлення, проте ці представлення не мали семантичного значення. Було б добре навчити векторне представлення таким чином, щоб схожі слова або синоніми відповідали векторами, які близькі один до одного за певною векторною відстанню (наприклад, евклідовою відстанню).

Для цього нам потрібно попередньо навчити модель вбудовування на великій колекції текстів, використовуючи техніку, таку як [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Вона базується на двох основних архітектурах, які використовуються для створення розподіленого представлення слів:

 - **Неперервний мішок слів** (CBoW), де ми навчаємо модель передбачати слово за навколишнім контекстом. Дано n-грам $(W_{-2},W_{-1},W_0,W_1,W_2)$, мета моделі — передбачити $W_0$ за $(W_{-2},W_{-1},W_1,W_2)$.
 - **Неперервний скіп-грам** є протилежністю CBoW. Модель використовує навколишнє вікно контекстних слів для передбачення поточного слова.

CBoW працює швидше, тоді як скіп-грам повільніший, але краще представляє рідковживані слова.

![Зображення, що показує алгоритми CBoW і Skip-Gram для перетворення слів у вектори.](../../../../../translated_images/uk/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Щоб експериментувати з вбудовуванням Word2Vec, попередньо навченим на наборі даних Google News, ми можемо використовувати бібліотеку **gensim**. Нижче наведено приклад пошуку слів, найбільш схожих до 'neural'.

> **Примітка:** Коли ви вперше створюєте векторні представлення слів, їх завантаження може зайняти деякий час!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Ми також можемо витягти векторне вбудовування зі слова, щоб використовувати його для навчання моделі класифікації. Вбудовування має 300 компонентів, але тут ми показуємо лише перші 20 компонентів вектора для ясності:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Чудова річ у семантичних вбудовуваннях полягає в тому, що ви можете маніпулювати векторним кодуванням на основі семантики. Наприклад, ми можемо попросити знайти слово, чия векторна репрезентація є якомога ближчою до слів *король* і *жінка*, і якомога далі від слова *чоловік*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Приклад вище використовує деяку внутрішню магію GenSym, але основна логіка насправді досить проста. Цікава річ про вбудовування полягає в тому, що ви можете виконувати звичайні операції з векторами на векторах вбудовування, і це буде відображати операції над **значеннями** слів. Приклад вище можна виразити в термінах векторних операцій: ми обчислюємо вектор, що відповідає **KING-MAN+WOMAN** (операції `+` і `-` виконуються на векторних представленнях відповідних слів), а потім знаходимо найближче слово в словнику до цього вектора:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Ми додали невеликі коефіцієнти до векторів *man* і *woman* - спробуйте їх прибрати, щоб побачити, що станеться.

Щоб знайти найближчий вектор, ми використовуємо механізми TensorFlow для обчислення вектора відстаней між нашим вектором і всіма векторами у словнику, а потім знаходимо індекс мінімального слова за допомогою `argmin`.


Хоча Word2Vec здається чудовим способом вираження семантики слів, він має багато недоліків, зокрема такі:

* Моделі CBoW і skip-gram є **прогнозуючими векторами**, і вони враховують лише локальний контекст. Word2Vec не використовує глобальний контекст.
* Word2Vec не враховує **морфологію** слів, тобто той факт, що значення слова може залежати від різних частин слова, таких як корінь.

**FastText** намагається подолати друге обмеження і розширює можливості Word2Vec, навчаючи векторні представлення для кожного слова та n-грам символів, які зустрічаються в кожному слові. Значення цих представлень потім усереднюються в один вектор на кожному етапі навчання. Хоча це додає багато додаткових обчислень до попереднього навчання, це дозволяє векторним представленням слів кодувати інформацію про частини слова.

Інший метод, **GloVe**, використовує інший підхід до векторних представлень слів, заснований на факторизації матриці "слово-контекст". Спочатку він створює велику матрицю, яка рахує кількість появ слова в різних контекстах, а потім намагається представити цю матрицю в нижчих вимірах таким чином, щоб мінімізувати втрати при реконструкції.

Бібліотека gensim підтримує ці векторні представлення слів, і ви можете експериментувати з ними, змінюючи код завантаження моделі вище.


## Використання попередньо навчених векторів у Keras

Ми можемо змінити наведений вище приклад, щоб заповнити матрицю в нашому шарі векторизації семантичними векторами, такими як Word2Vec. Словники попередньо навчених векторів і текстового корпусу, ймовірно, не співпадатимуть, тому нам потрібно обрати один. Тут ми розглянемо два можливі варіанти: використання словника токенізатора та використання словника з Word2Vec.

### Використання словника токенізатора

При використанні словника токенізатора деякі слова зі словника матимуть відповідні вектори Word2Vec, а деякі будуть відсутні. Враховуючи, що розмір нашого словника дорівнює `vocab_size`, а довжина векторів Word2Vec — `embed_size`, шар векторизації буде представлений матрицею ваг розміром `vocab_size`$\times$`embed_size`. Ми заповнимо цю матрицю, проходячи через словник:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Для слів, які відсутні у словнику Word2Vec, ми можемо залишити їх як нулі або згенерувати випадковий вектор.

Тепер ми можемо визначити шар вбудовування з попередньо навченими вагами:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Примітка**: Зверніть увагу, що ми встановили `trainable=False` під час створення `Embedding`, що означає, що ми не перенавчаємо шар Embedding. Це може трохи знизити точність, але прискорює навчання.

### Використання словника ембедингів

Одна з проблем попереднього підходу полягає в тому, що словники, які використовуються в TextVectorization і Embedding, відрізняються. Щоб вирішити цю проблему, ми можемо скористатися одним із наступних рішень:
* Перенавчити модель Word2Vec на нашому словнику.
* Завантажити наш набір даних із використанням словника з попередньо навченої моделі Word2Vec. Словники, які використовуються для завантаження набору даних, можна вказати під час завантаження.

Другий підхід здається простішим, тому давайте реалізуємо його. Спочатку ми створимо шар `TextVectorization` із вказаним словником, взятим із ембедингів Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Бібліотека векторних представлень слів gensim містить зручну функцію `get_keras_embeddings`, яка автоматично створить відповідний шар векторних представлень Keras для вас.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Однією з причин, чому ми не спостерігаємо вищої точності, є те, що деякі слова з нашого набору даних відсутні у попередньо натренованому словнику GloVe, і тому вони фактично ігноруються. Щоб вирішити це, ми можемо натренувати власні векторні представлення на основі нашого набору даних.


## Контекстуальні вбудовування

Одним із ключових обмежень традиційних попередньо навчених представлень вбудовувань, таких як Word2Vec, є те, що, хоча вони можуть передавати певне значення слова, вони не здатні розрізняти різні значення. Це може спричиняти проблеми в моделях, які використовуються далі.

Наприклад, слово «play» має різні значення в цих двох реченнях:
- Я ходив на **виставу** в театрі.
- Джон хоче **гратися** зі своїми друзями.

Попередньо навчені вбудовування, про які ми говорили, представляють обидва значення слова «play» в одному вбудовуванні. Щоб подолати це обмеження, нам потрібно створювати вбудовування на основі **мовної моделі**, яка тренується на великому корпусі тексту і *знає*, як слова можуть поєднуватися в різних контекстах. Обговорення контекстуальних вбудовувань виходить за рамки цього уроку, але ми повернемося до них, коли будемо говорити про мовні моделі в наступному розділі.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
